<a href="https://colab.research.google.com/github/lakshmansamvith/Digit-Recognizer/blob/master/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow 

url1 = 'https://raw.githubusercontent.com/lakshmansamvith/Digit-Recognizer/master/train.csv'
url2 = 'https://raw.githubusercontent.com/lakshmansamvith/Digit-Recognizer/master/test.csv'
training_data_frame = pd.read_csv(url1)
testing_data_frame = pd.read_csv(url2)
training_data_frame

#Checking for Null Values in Columns 

In [ ]:
testing_data_frame.isnull().any()
sum(testing_data_frame.isnull().sum())
training_data_frame.isnull().any()
sum(training_data_frame.isnull().sum())
#sns.countplot(training_data_frame['label'])

#Preprocessing Data

In [ ]:
y_df = pd.DataFrame(training_data_frame[['label']].values.reshape(len(training_data_frame),))
X_df = pd.DataFrame(training_data_frame.drop(labels = ["label"],axis = 1))
print(type(X_df))
y_df

#One Hot Encoding

In [ ]:
from keras.utils.np_utils import to_categorical
y= to_categorical(y_df)

#Reshaping Data and Splitting Data 

In [ ]:
from sklearn.model_selection import train_test_split

X = X_df.values.reshape(X_df.shape[0], 28, 28, 1)
y = y_df.values.reshape(len(y),)
testing_data = testing_data_frame.values.reshape(testing_data_frame.shape[0], 28, 28, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#Print all the shapes(Reference)

In [ ]:
type(X), type(y), X.shape, X_train.shape, y.shape , y_train.shape


#Creating a Convolution Network 

In [ ]:
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Dropout, Flatten
from tensorflow.keras.models import Sequential 

model = Sequential([Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)), 
                   MaxPooling2D(2, 2),
                   Conv2D(32, (3,3), activation='relu'),
                   MaxPooling2D(2, 2), 
                   Flatten(),
                   Dense(1024, activation='relu' ),
                   Dropout(0.1),
                   Dense(1024, activation='relu' ),
                   Dropout(0.1), 
                   Dense(10, activation='softmax'),
                   ])

#Data Augmentation 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

training_generator= training_datagen.flow(X_train, y_train, 
                                          batch_size=64,
                                          )

validation_generator = validation_datagen.flow(X_test, y_test, 
                                               batch_size=64,
                                               ) 


# Model Summary

In [ ]:
model.summary()

#CallBack function 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

class myCallback(tensorflow.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('val_accuracy')>0.995):
                print("\nReached 99.5% accuracy so cancelling training!")
                self.model.stop_training = True

checkpt =  ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

#Model Compilation and Model Fitting 

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(), metrics=['accuracy'], loss='sparse_categorical_crossentropy')

callback = myCallback()

history = model.fit(training_generator, 
                               epochs=100, 
                               validation_data=validation_generator, 
                               verbose=2,
                              batch_size=64,
                               steps_per_epoch=(X_train.shape[0])//64, 
                              callbacks=[callback, checkpt])

# Plotting for understanding

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

# Predicting on Test Data

In [ ]:
test_values = model.predict(testing_data)
predicted_numbers = [np.argmax(number) for number in test_values]


#Saving to CSV File

In [ ]:
import csv 
with open('digit.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['ImageId', 'Label'])
    for count in range(0, 28000): 
            writer.writerow((count+1, predicted_numbers[count]))
            row = row + 1

        

In [ ]:
predicted_numbers[27999]